In [1]:
import os
import numpy as np
import pandas as pd
import shutil
import baostock as bs
import talib
import tushare as ts

## 策略

In [2]:
class DataLoader:
    def __init__(self, 
                 stock_data_path, 
                 hs300_constituents_path):
        self.stock_data_path = stock_data_path
        self.hs300_constituents_path = hs300_constituents_path
        
    def load_hs300_constituents(self):
        """Load HS300 constituent stocks data from CSV file"""
        if not os.path.exists(self.hs300_constituents_path):
            raise FileNotFoundError(f"HS300 constituents file not found: {self.hs300_constituents_path}")
            
        df = pd.read_csv(self.hs300_constituents_path)
        df['updateDate'] = pd.to_datetime(df['updateDate'])
        return df
        
    def load_stock_data(self):
        """Load stock price data from CSV file"""
        if not os.path.exists(self.stock_data_path):
            raise FileNotFoundError(f"Stock data file not found: {self.stock_data_path}")
            
        df = pd.read_csv(self.stock_data_path)
        df['date'] = pd.to_datetime(df['date'])
        df = df.sort_values(['code', 'date'])
        return df 

In [3]:
class TechnicalAnalysis:
    @staticmethod
    def calculate_ma(df, window=20):
        """Calculate Moving Average for each stock"""
        return df.groupby('code')['close'].transform(lambda x: x.rolling(window=window).mean())
    
    @staticmethod
    def calculate_kdj(df, n=9, m1=3, m2=3):
        """
        Calculate KDJ indicator
        n: RSV period
        m1: K period
        m2: D period
        """
        df = df.copy()
        
        # Group by code to calculate KDJ for each stock
        for code in df['code'].unique():
            mask = df['code'] == code
            df_stock = df[mask].copy()
            
            df_stock = df_stock.reset_index(drop=True)
            
            # Calculate RSV
            low_list = df_stock['low'].rolling(window=n, min_periods=1).min()
            high_list = df_stock['high'].rolling(window=n, min_periods=1).max()
            rsv = (df_stock['close'] - low_list) / (high_list - low_list) * 100
            
            # Initialize K, D, J arrays
            k = np.zeros(len(df_stock))
            d = np.zeros(len(df_stock))
            
            # Calculate K and D
            for i in range(len(df_stock)):
                if i == 0:
                    k[i] = 50
                    d[i] = 50
                else:
                    k[i] = (m1 - 1) * k[i-1] / m1 + rsv[i] / m1
                    d[i] = (m2 - 1) * d[i-1] / m2 + k[i] / m2
            
            # Calculate J
            j = 3 * k - 2 * d
            
            df.loc[mask, 'kdj_k'] = k
            df.loc[mask, 'kdj_d'] = d
            df.loc[mask, 'kdj_j'] = j
            
        return df 

In [4]:
class TradingStrategy:
    def __init__(self):
        self.ta = TechnicalAnalysis()
        
    def prepare_data(self, df):
        """Prepare data by calculating necessary indicators"""
        df = df.copy()
        # Calculate MA5
        df['ma5'] = self.ta.calculate_ma(df, window=5)
        # Calculate MA10
        df['ma20'] = self.ta.calculate_ma(df, window=20)
        # df['ma10'] = self.ta.calculate_ma(df, window=10)
        # Calculate MA60
        df['ma60'] = self.ta.calculate_ma(df, window=60)
        # Calculate KDJ
        df = self.ta.calculate_kdj(df)
        return df
        
    def find_trading_signals(self, df):
        """
        Find trading signals based on strategy rules:
        1. Price above MA20
        2. Find where J turns negative (D1) and then turns positive (D2)
        """
        df = df.copy()
        
        # Check if price is above MA10
        df['above_ma20'] = df['close'] > df['ma20']
        
        # Find where J value turns negative and then positive
        df['prev_j'] = df.groupby('code')['kdj_j'].shift(1)
        df['j_turns_negative'] = (df['kdj_j'] < 0) & (df['prev_j'] >= 0)
        
        # Create initial signals when J turns negative (D1)
        signals = df[df['above_ma20'] & df['j_turns_negative']].copy()
        
        # For each D1 signal, find the corresponding D2 (when J turns positive)
        results = []
        for _, signal in signals.iterrows():
            code = signal['code']
            d1_date = signal['date']
            
            # Find the next date when J turns positive after D1
            future_data = df[
                (df['code'] == code) & 
                (df['date'] > d1_date)
            ].copy()
            
            future_data['j_turns_positive'] = (future_data['kdj_j'] >= 0) & (future_data['prev_j'] < 0)
            d2_data = future_data[future_data['j_turns_positive']]
            
            if len(d2_data) > 0:
                d2_date = d2_data.iloc[0]['date']
                d2_price = d2_data.iloc[0]['close']
                period_return = (d2_price / signal['close'] - 1) * 100
                
                result = {
                    'code': code,
                    'D1日期': d1_date,
                    'D2日期': d2_date,
                    'D1收盘价': signal['close'],
                    'D2收盘价': d2_price,
                    'D1-D2收益率': period_return,
                    'D1_5日均线': signal['ma5'],
                    'D1_20日均线': signal['ma20'],
                    'D1_60日均线': signal['ma60'],
                    'D1_J值': signal['kdj_j'],
                    'D2_J值': d2_data.iloc[0]['kdj_j'],
                    '持仓天数': (d2_date - d1_date).days
                }
                results.append(result)
        
        if not results:
            return pd.DataFrame()
            
        results_df = pd.DataFrame(results)
        return results_df
        
    def calculate_returns(self, df, signals, days=10):
        """This method is kept for backward compatibility but not used in the new strategy"""
        return pd.DataFrame() 

## 沪深300股票id

In [5]:
"""沪深300成分股"""
lg = bs.login()
if lg.error_code != '0':
    print(f'登录失败，错误代码：{lg.error_code}, 错误信息：{lg.error_msg}')
else:
    print('登录成功')

# 获取沪深300成分股
rs = bs.query_hs300_stocks()
if rs.error_code != '0':
    print(f'查询沪深300成分股失败，错误代码：{rs.error_code}, 错误信息：{rs.error_msg}')
else:
    # 保存成DataFrame
    hs300_stocks = []
    while rs.next():
        hs300_stocks.append(rs.get_row_data())
    df_hs300 = pd.DataFrame(hs300_stocks, columns=rs.fields)
    print(df_hs300)

bs.logout()

login success!
登录成功
     updateDate       code code_name
0    2025-04-21  sh.600000      浦发银行
1    2025-04-21  sh.600009      上海机场
2    2025-04-21  sh.600010      包钢股份
3    2025-04-21  sh.600011      华能国际
4    2025-04-21  sh.600015      华夏银行
..          ...        ...       ...
295  2025-04-21  sz.300832       新产业
296  2025-04-21  sz.300896       爱美客
297  2025-04-21  sz.300979      华利集团
298  2025-04-21  sz.300999       金龙鱼
299  2025-04-21  sz.301269      华大九天

[300 rows x 3 columns]
logout success!


In [6]:
df_hs300

,updateDate,code,code_name
0,2025-04-21,sh.600000,浦发银行
1,2025-04-21,sh.600009,上海机场
2,2025-04-21,sh.600010,包钢股份
3,2025-04-21,sh.600011,华能国际
4,2025-04-21,sh.600015,华夏银行
...,...,...,...
295,2025-04-21,sz.300832,新产业
296,2025-04-21,sz.300896,爱美客
297,2025-04-21,sz.300979,华利集团
298,2025-04-21,sz.300999,金龙鱼


In [7]:
df_hs300[df_hs300["code_name"] == "赛力斯"]

,updateDate,code,code_name
94,2025-04-21,sh.601127,赛力斯


In [10]:
df_hs300[df_hs300["code_name"] == "五粮液"]

,updateDate,code,code_name
218,2025-04-21,sz.000858,五粮液


In [11]:
df_hs300[df_hs300["code_name"] == "宁德时代"]

,updateDate,code,code_name
291,2025-04-21,sz.300750,宁德时代


In [12]:
df_hs300[df_hs300["code_name"] == "比亚迪"]

,updateDate,code,code_name
259,2025-04-21,sz.002594,比亚迪


In [13]:
df_hs300[df_hs300["code_name"] == "恒生电子"]

,updateDate,code,code_name
54,2025-04-21,sh.600570,恒生电子


## 获取行情

In [16]:
"""
23年
"""
lg = bs.login()
if lg.error_code != '0':
    print(f'登录失败，错误代码：{lg.error_code}, 错误信息：{lg.error_msg}')
else:
    print('登录成功')

# 设置日期范围
start_date = '2023-01-01'
end_date = '2023-12-31'

# 初始化存储数据的列表
stock_data_2023 = []

# 遍历每只成分股
for code in df_hs300['code']:
    # 获取日K线数据
    rs = bs.query_history_k_data_plus(
        code,
        "date,code,open,high,low,close,preclose,volume,amount,turn",
        start_date=start_date, end_date=end_date,
        frequency="d", adjustflag="3"
    )
    if rs.error_code != '0':
        print(f'查询股票 {code} 数据失败，错误代码：{rs.error_code}, 错误信息：{rs.error_msg}')
        continue

    # 保存数据
    stock_data = []
    while rs.next():
        stock_data.append(rs.get_row_data())
    df_stock = pd.DataFrame(stock_data, columns=rs.fields)
    stock_data_2023.append(df_stock)

# 合并所有股票的数据
df_stock_2023 = pd.concat(stock_data_2023, ignore_index=True)

# 登出系统
bs.logout()

login success!
登录成功
logout success!


In [19]:
df_stock_2023.to_csv("/home/kennys/MineX/QuantTrading/dataset/沪深300-2023年数据.csv")

In [17]:
df_stock_2023

,date,code,open,high,low,close,preclose,volume,amount,turn
0,2023-01-03,sh.600000,7.2700,7.2800,7.1700,7.2300,7.2800,25892521,187094064.4100,0.088200
1,2023-01-04,sh.600000,7.2700,7.3500,7.2300,7.3100,7.2300,30947081,226321372.0500,0.105400
2,2023-01-05,sh.600000,7.3700,7.3800,7.3000,7.3500,7.3100,30162154,221617354.6200,0.102800
3,2023-01-06,sh.600000,7.3500,7.3800,7.3100,7.3400,7.3500,20312881,149170537.8500,0.069200
4,2023-01-09,sh.600000,7.3800,7.3800,7.3000,7.3400,7.3400,19612260,143998210.6000,0.066800
...,...,...,...,...,...,...,...,...,...,...
72328,2023-12-25,sz.301269,101.1200,102.9800,100.2200,102.6900,101.5000,1306118,133157675.1500,0.488400
72329,2023-12-26,sz.301269,102.6800,104.2000,101.4000,103.3300,102.6900,1398825,144208092.2200,0.523000
72330,2023-12-27,sz.301269,103.5000,105.3800,102.7000,104.4400,103.3300,1710496,178485254.8100,0.639600
72331,2023-12-28,sz.301269,104.0000,105.3000,103.3000,105.1900,104.4400,2035357,212350969.3200,0.761000


In [12]:
"""
24年至今
"""
# 登录系统
lg = bs.login()
if lg.error_code != '0':
    print(f'登录失败，错误代码：{lg.error_code}, 错误信息：{lg.error_msg}')
else:
    print('登录成功')

# 设置日期范围
start_date = '2024-01-01'
end_date = '2025-04-22'

# 初始化存储数据的列表
all_stock_data = []

# 遍历每只成分股
for code in df_hs300['code']:
    # 获取日K线数据
    rs = bs.query_history_k_data_plus(
        code,
        "date,code,open,high,low,close,preclose,volume,amount,turn",
        start_date=start_date, end_date=end_date,
        frequency="d", adjustflag="3"
    )
    if rs.error_code != '0':
        print(f'查询股票 {code} 数据失败，错误代码：{rs.error_code}, 错误信息：{rs.error_msg}')
        continue

    # 保存数据
    stock_data = []
    while rs.next():
        stock_data.append(rs.get_row_data())
    df_stock = pd.DataFrame(stock_data, columns=rs.fields)
    all_stock_data.append(df_stock)

# 合并所有股票的数据
df_all_stocks = pd.concat(all_stock_data, ignore_index=True)

# 登出系统
bs.logout()

login success!
登录成功
logout success!


In [18]:
df_all_stocks.to_csv("/home/kennys/MineX/QuantTrading/dataset/沪深300-2024年至今数据.csv")

In [15]:
df_all_stocks

,date,code,open,high,low,close,preclose,volume,amount,turn
0,2024-01-02,sh.600000,6.6300,6.6500,6.6000,6.6000,6.6200,22066700,146066303.7200,0.075200
1,2024-01-03,sh.600000,6.5900,6.6500,6.5900,6.6400,6.6000,18203654,120639706.0100,0.062000
2,2024-01-04,sh.600000,6.6400,6.6700,6.5500,6.6200,6.6400,28885978,190580609.9900,0.098400
3,2024-01-05,sh.600000,6.6000,6.7600,6.5900,6.6800,6.6200,44421387,296976885.7900,0.151300
4,2024-01-08,sh.600000,6.6800,6.7100,6.5600,6.5900,6.6800,37520337,247977824.9800,0.127800
...,...,...,...,...,...,...,...,...,...,...
94195,2025-04-16,sz.301269,124.6800,125.5500,121.8000,123.8000,123.5400,6258128,774022704.8500,2.385700
94196,2025-04-17,sz.301269,122.3500,125.5000,122.3500,123.1800,123.8000,4504425,558238259.6700,1.717100
94197,2025-04-18,sz.301269,122.5000,125.2800,120.3400,120.9800,123.1800,4141438,505640958.5400,1.578800
94198,2025-04-21,sz.301269,120.8000,124.6000,119.8100,123.9100,120.9800,4083840,499746858.2000,1.556800


In [7]:
"""
25年
"""
lg = bs.login()
if lg.error_code != '0':
    print(f'登录失败，错误代码：{lg.error_code}, 错误信息：{lg.error_msg}')
else:
    print('登录成功')

# 设置日期范围
start_date = '2025-01-01'
end_date = '2025-4-26'

# 初始化存储数据的列表
stock_data_2025 = []

# 遍历每只成分股
for code in df_hs300['code']:
    # 获取日K线数据
    rs = bs.query_history_k_data_plus(
        code,
        "date,code,open,high,low,close,preclose,volume,amount,turn",
        start_date=start_date, end_date=end_date,
        frequency="d", adjustflag="3"
    )
    if rs.error_code != '0':
        print(f'查询股票 {code} 数据失败，错误代码：{rs.error_code}, 错误信息：{rs.error_msg}')
        continue

    # 保存数据
    stock_data = []
    while rs.next():
        stock_data.append(rs.get_row_data())
    df_stock = pd.DataFrame(stock_data, columns=rs.fields)
    stock_data_2025.append(df_stock)

# 合并所有股票的数据
df_stock_2025 = pd.concat(stock_data_2025, ignore_index=True)

# 登出系统
bs.logout()

login success!
登录成功
logout success!


In [8]:
df_stock_2025.to_csv("/home/kennys/experiment/QuantTrading/dataset/沪深300-2025年至今数据.csv")

In [10]:
df_stock_2025

,date,code,open,high,low,close,preclose,volume,amount,turn
0,2025-01-02,sh.600000,10.3000,10.4200,10.0500,10.1300,10.2900,78907129,803904040.2300,0.268800
1,2025-01-03,sh.600000,10.1200,10.2200,10.0200,10.0600,10.1300,50693944,511576493.6000,0.172700
2,2025-01-06,sh.600000,10.1300,10.2000,9.8100,10.1500,10.0600,79801276,804593503.5600,0.271900
3,2025-01-07,sh.600000,10.1100,10.3000,10.1100,10.2700,10.1500,40265146,411488491.5100,0.137200
4,2025-01-08,sh.600000,10.2600,10.3700,10.2000,10.3000,10.2700,48237450,496301044.8100,0.164300
...,...,...,...,...,...,...,...,...,...,...
22195,2025-04-18,sz.301269,122.5000,125.2800,120.3400,120.9800,123.1800,4141438,505640958.5400,1.578800
22196,2025-04-21,sz.301269,120.8000,124.6000,119.8100,123.9100,120.9800,4083840,499746858.2000,1.556800
22197,2025-04-22,sz.301269,123.0100,124.5000,121.6100,122.8000,123.9100,3159912,389129941.6000,1.204600
22198,2025-04-23,sz.301269,123.0000,123.4800,120.2000,121.2000,122.8000,3464362,420032149.6200,1.320700


In [23]:
df_stock_2025[df_stock_2025["code"] == "sh.601888"]

,date,code,open,high,low,close,preclose,volume,amount,turn
10725,2025-01-02,sh.601888,67.0300,67.1900,64.3300,64.7000,67.0100,18671166,1227701232.9100,0.956300
10726,2025-01-03,sh.601888,64.9900,65.2000,62.7600,62.8300,64.7000,15383769,979895136.3000,0.787900
10727,2025-01-06,sh.601888,62.8300,63.2900,62.1500,62.5400,62.8300,9595563,601081039.5800,0.491500
10728,2025-01-07,sh.601888,62.5400,62.7100,61.8600,62.4700,62.5400,9616460,599735313.0600,0.492500
10729,2025-01-08,sh.601888,62.4700,62.4700,60.6600,61.7900,62.4700,13764819,846538814.2900,0.705000
10730,2025-01-09,sh.601888,61.6000,61.9900,60.8200,61.0100,61.7900,12824021,786838965.3900,0.656800
10731,2025-01-10,sh.601888,60.8000,61.0000,59.9300,59.9800,61.0100,11265146,680046800.9800,0.577000
10732,2025-01-13,sh.601888,59.5000,60.6500,59.3700,60.1700,59.9800,9198864,552403606.9800,0.471100
10733,2025-01-14,sh.601888,60.2000,61.9800,60.2000,61.8700,60.1700,16542556,1012419547.4400,0.847300
10734,2025-01-15,sh.601888,61.7900,61.7900,61.0500,61.2900,61.8700,9372462,574830484.7900,0.480000


### 计算个股

In [28]:
def calculate_kdj(data, n=9, m1=3, m2=3):
    """
    计算KDJ指标
    n: RSV周期，默认9
    m1: K值周期，默认3
    m2: D值周期，默认3
    """
    data = data.copy()
    
    # 计算RSV
    low_list = data['low'].rolling(window=n, min_periods=1).min()
    high_list = data['high'].rolling(window=n, min_periods=1).max()
    rsv = (data['close'] - low_list) / (high_list - low_list) * 100
    
    # 初始化K、D值
    k = np.zeros(len(data))
    d = np.zeros(len(data))
    
    # 计算K、D值
    for i in range(len(data)):
        if i == 0:
            k[i] = 50
            d[i] = 50
        else:
            k[i] = (m1 - 1) * k[i-1] / m1 + rsv[i] / m1
            d[i] = (m2 - 1) * d[i-1] / m2 + k[i] / m2
    
    # 计算J值
    j = 3 * k - 2 * d
    
    return pd.Series(j)

In [29]:
df_stock = df_stock_2025[df_stock_2025["code"] == "sh.601888"].copy()
df_stock = df_stock.sort_values('date').reset_index(drop=True)

# 2. 将价格相关的列转换为浮点数
numeric_columns = ['open', 'high', 'low', 'close', 'volume']
for col in numeric_columns:
    df_stock[col] = pd.to_numeric(df_stock[col], errors='coerce')

df_stock['kdj_j'] = calculate_kdj(df_stock)

# 显示结果
print("\n=== 中国中免(sh.601888) KDJ指标J值 ===")
print(df_stock[['date', 'close', 'kdj_j']].to_string())



=== 中国中免(sh.601888) KDJ指标J值 ===
          date  close       kdj_j
0   2025-01-02  64.70   50.000000
1   2025-01-03  62.83   12.340105
2   2025-01-06  62.54   -0.803654
3   2025-01-07  62.47   -2.813363
4   2025-01-08  61.79    2.435508
5   2025-01-09  61.01   -2.869977
6   2025-01-10  59.98   -7.946676
7   2025-01-13  60.17   -1.822353
8   2025-01-14  61.87   19.146950
9   2025-01-15  61.29   30.640966
10  2025-01-16  61.60   54.635494
11  2025-01-17  61.02   59.727102
12  2025-01-20  61.88   85.191981
13  2025-01-21  61.53   85.251383
14  2025-01-22  60.38   55.543970
15  2025-01-23  60.52   43.014669
16  2025-01-24  61.25   46.324035
17  2025-01-27  60.70   31.536638
18  2025-02-05  60.22   10.092870
19  2025-02-06  60.95   34.379560
20  2025-02-07  61.64   62.706351
21  2025-02-10  62.05   85.591143
22  2025-02-11  60.97   74.727713
23  2025-02-12  61.48   80.429427
24  2025-02-13  61.99   92.319439
25  2025-02-14  62.09   96.078240
26  2025-02-17  61.75   82.923902
27  2025-02-18 

## 计算

In [30]:
"""
J值为负
"""
selected_df = pd.DataFrame(pd.read_csv("/home/kennys/experiment/QuantTrading/core/ma20-20250427.csv"))
selected_df

,code,code_name,信号日期,当日收盘价,5日均线,20日均线,60日均线,当日J值,前一日J值,5日收益率,10日收益率,30日收益率
0,sh.600104,上汽集团,2025-01-06,17.87,NaN,NaN,NaN,-3.69,11.41,-8.95,-4.36,-4.76
1,sh.600372,中航机载,2025-01-06,11.40,NaN,NaN,NaN,-0.49,12.94,-1.49,1.40,2.46
2,sh.600515,海南机场,2025-01-06,3.49,NaN,NaN,NaN,-1.22,13.89,0.00,8.02,4.87
3,sh.600570,恒生电子,2025-01-06,25.16,NaN,NaN,NaN,-1.74,12.66,0.48,4.09,30.09
4,sh.600584,长电科技,2025-01-06,35.66,NaN,NaN,NaN,-2.05,13.27,9.09,16.71,12.68
5,sh.600745,闻泰科技,2025-01-06,31.64,NaN,NaN,NaN,-4.10,11.25,-0.98,6.98,17.95
6,sh.601136,首创证券,2025-01-06,19.97,NaN,NaN,NaN,-3.00,13.53,-3.30,-0.80,6.26
7,sh.601336,新华保险,2025-01-06,45.68,NaN,NaN,NaN,-0.09,12.77,-1.82,4.20,8.36
8,sh.601888,中国中免,2025-01-06,62.54,NaN,NaN,NaN,-0.80,12.34,-3.79,-1.06,-3.98
9,sh.603019,中科曙光,2025-01-06,64.51,NaN,NaN,NaN,-1.40,11.53,-5.61,0.34,17.97


In [32]:
april_df = selected_df[selected_df['信号日期'].str.startswith('2025-04')]
april_df

,code,code_name,信号日期,当日收盘价,5日均线,20日均线,60日均线,当日J值,前一日J值,5日收益率,10日收益率,30日收益率
676,sh.600029,南方航空,2025-04-01,5.71,5.87,5.92,NaN,-0.98,2.13,0.35,2.45,NaN
677,sh.600031,三一重工,2025-04-01,18.93,19.16,19.80,NaN,-1.35,0.38,1.00,0.90,NaN
678,sh.600188,兖矿能源,2025-04-01,13.44,13.64,13.53,NaN,-2.89,1.96,-5.88,-4.24,NaN
679,sh.600690,海尔智家,2025-04-01,26.95,27.49,27.30,NaN,-11.13,3.15,-7.09,-7.27,NaN
680,sh.600958,东方证券,2025-04-01,9.38,9.60,9.69,NaN,-4.19,4.81,-5.33,-2.35,NaN
681,sh.601390,中国中铁,2025-04-01,5.56,5.73,5.81,NaN,-5.39,1.38,2.16,2.70,NaN
682,sh.601633,长城汽车,2025-04-01,25.70,26.24,26.09,NaN,-3.66,0.87,-10.51,-9.18,NaN
683,sh.601658,邮储银行,2025-04-01,5.16,5.25,5.31,NaN,-3.25,6.56,-1.36,2.13,NaN
684,sh.601669,中国电建,2025-04-01,4.81,4.87,4.89,NaN,-0.68,4.44,-0.21,-0.62,NaN
685,sh.601766,中国中车,2025-04-01,6.98,7.23,7.37,NaN,-0.37,4.93,2.44,0.72,NaN


In [20]:
"""
J值由负转正
"""
turned_df = pd.DataFrame(pd.read_csv("/home/kennys/experiment/QuantTrading/testdata/temp/turned-20250427.csv"))
turned_df

,code,code_name,D1日期,D2日期,D1收盘价,D2收盘价,D1-D2收益率,D1_5日均线,D1_20日均线,D1_60日均线,D1_J值,D2_J值,D1_WR14,D2_WR14,D1_WR28,D2_WR28,持仓天数
0,sh.600031,三一重工,2025-03-21,2025-03-24,19.59,19.96,1.89,19.96,19.40,NaN,-12.22,5.34,-52.02,-50.97,-29.07,-20.52,3
1,sh.600039,四川路桥,2025-03-31,2025-04-01,7.94,8.00,0.76,8.08,7.86,NaN,-2.61,3.86,-55.00,-60.32,-29.33,-27.54,1
2,sh.600050,中国联通,2025-03-03,2025-03-06,6.22,6.53,4.98,6.42,5.97,NaN,-9.64,26.33,-46.71,-47.06,-32.87,-18.52,3
3,sh.600160,巨化股份,2025-03-20,2025-03-25,25.03,23.90,-4.51,25.32,24.88,NaN,-1.12,2.52,-52.65,-82.08,-40.88,-74.12,5
4,sh.600161,天坛生物,2025-02-13,2025-02-14,20.31,20.67,1.77,20.63,20.19,NaN,-12.86,10.68,-43.24,-31.41,-40.17,-25.10,1
5,sh.600406,国电南瑞,2025-04-22,2025-04-24,23.06,23.13,0.30,23.35,22.87,22.98,-5.15,11.46,-47.60,-45.51,-47.60,-45.51,2
6,sh.600570,恒生电子,2025-02-27,2025-03-04,32.37,30.95,-4.39,33.46,31.39,NaN,-2.29,0.96,-65.26,-87.50,-37.93,-55.04,5
7,sh.600588,用友网络,2025-02-27,2025-03-03,16.08,16.40,1.99,16.69,15.20,NaN,-5.73,13.70,-58.02,-60.52,-32.36,-29.46,4
8,sh.601006,大秦铁路,2025-02-25,2025-02-26,6.65,6.73,1.20,6.75,6.62,NaN,-6.14,5.37,-63.46,-51.02,-39.76,-30.12,1
9,sh.601166,兴业银行,2025-03-27,2025-04-03,21.50,21.18,-1.49,21.64,21.46,NaN,-8.41,4.75,-63.64,-87.25,-45.58,-60.47,7


In [21]:
"""
J值由负转正 April数据
"""
turned_april_df = turned_df[turned_df['D1日期'].str.startswith('2025-04')]
turned_april_df

,code,code_name,D1日期,D2日期,D1收盘价,D2收盘价,D1-D2收益率,D1_5日均线,D1_20日均线,D1_60日均线,D1_J值,D2_J值,D1_WR14,D2_WR14,D1_WR28,D2_WR28,持仓天数
5,sh.600406,国电南瑞,2025-04-22,2025-04-24,23.06,23.13,0.3,23.35,22.87,22.98,-5.15,11.46,-47.60,-45.51,-47.60,-45.51,2
25,sz.000876,新希望,2025-04-21,2025-04-22,10.05,10.04,-0.1,10.06,9.91,9.54,-0.27,2.24,-70.11,-71.32,-68.54,-68.91,1


### 计算全面指标

In [24]:
"""
计算全面指标
"""
import talib

# 1. 获取股票数据
stock_data = df_stock_2025[df_stock_2025["code"] == "sh.601888"].copy()
stock_data["date"] = pd.to_datetime(stock_data["date"])
stock_data = stock_data.sort_values('date').reset_index(drop=True)  # 重置索引，确保索引连续

# 2. 将价格相关的列转换为浮点数
numeric_columns = ['open', 'high', 'low', 'close', 'preclose', 'volume', 'amount', 'turn']
for col in numeric_columns:
    stock_data[col] = pd.to_numeric(stock_data[col])

# 3. 计算MACD指标
macd, macd_signal, macd_hist = talib.MACD(
    stock_data['close'],
    fastperiod=12,
    slowperiod=26,
    signalperiod=9
)

# 4. 计算KDJ指标
def calculate_kdj(data, n=9, m1=3, m2=3):
    data = data.copy()
    
    # 计算RSV
    low_list = data['low'].rolling(window=n, min_periods=1).min()
    high_list = data['high'].rolling(window=n, min_periods=1).max()
    rsv = (data['close'] - low_list) / (high_list - low_list) * 100
    
    # 计算K、D、J值
    k = np.zeros(len(data))
    d = np.zeros(len(data))
    
    # 初始化第一个值
    k[0] = 50
    d[0] = 50
    
    # 计算其他值
    for i in range(1, len(data)):
        k[i] = (m1 - 1) * k[i-1] / m1 + rsv[i] / m1
        d[i] = (m2 - 1) * d[i-1] / m2 + k[i] / m2
    
    # 计算J值
    j = 3 * k - 2 * d
    
    return pd.Series(k), pd.Series(d), pd.Series(j)

# 计算KDJ
k, d, j = calculate_kdj(stock_data)

# 5. 计算BOLL指标
upper, middle, lower = talib.BBANDS(
    stock_data['close'],
    timeperiod=20,
    nbdevup=2,
    nbdevdn=2,
    matype=0
)

# 6. 将计算结果添加到数据框中
stock_data['MACD'] = macd
stock_data['MACD_signal'] = macd_signal
stock_data['MACD_hist'] = macd_hist
stock_data['KDJ_K'] = k
stock_data['KDJ_D'] = d
stock_data['KDJ_J'] = j
stock_data['BOLL_upper'] = upper
stock_data['BOLL_middle'] = middle
stock_data['BOLL_lower'] = lower

# 7. 获取2025/4/23的指标值
target_date = '2025-04-25'
result = stock_data[stock_data['date'] == target_date]

if len(result) == 0:
    print(f"未找到 {target_date} 的数据")
else:
    result = result.iloc[0]
    
    # 8. 打印结果
    print(f"\n=== {result['code']} ({target_date}) 技术指标分析 ===")
    print("\nMACD指标:")
    print(f"MACD值: {result['MACD']:.4f}")
    print(f"MACD信号线: {result['MACD_signal']:.4f}")
    print(f"MACD柱状图: {result['MACD_hist']:.4f}")

    print("\nKDJ指标:")
    print(f"K值: {result['KDJ_K']:.4f}")
    print(f"D值: {result['KDJ_D']:.4f}")
    print(f"J值: {result['KDJ_J']:.4f}")

    print("\nBOLL指标:")
    print(f"上轨: {result['BOLL_upper']:.4f}")
    print(f"中轨: {result['BOLL_middle']:.4f}")
    print(f"下轨: {result['BOLL_lower']:.4f}")

    # 9. 简单的技术分析
    print("\n=== 技术分析 ===")

    # MACD分析
    if result['MACD_hist'] > 0:
        macd_signal = "MACD处于上升趋势"
    else:
        macd_signal = "MACD处于下降趋势"
    print(f"MACD分析: {macd_signal}")

    # KDJ分析
    if result['KDJ_J'] > 100:
        kdj_signal = "KDJ超买"
    elif result['KDJ_J'] < 0:
        kdj_signal = "KDJ超卖"
    else:
        if result['KDJ_J'] > result['KDJ_K'] and result['KDJ_K'] > result['KDJ_D']:
            kdj_signal = "KDJ金叉，看多信号"
        elif result['KDJ_J'] < result['KDJ_K'] and result['KDJ_K'] < result['KDJ_D']:
            kdj_signal = "KDJ死叉，看空信号"
        else:
            kdj_signal = "KDJ处于盘整区间"
    print(f"KDJ分析: {kdj_signal}")

    # BOLL分析
    close = result['close']
    if close > result['BOLL_upper']:
        boll_signal = "股价突破布林上轨，超买区间"
    elif close < result['BOLL_lower']:
        boll_signal = "股价突破布林下轨，超卖区间"
    else:
        boll_signal = "股价在布林带中轨运行，趋势盘整"
    print(f"BOLL分析: {boll_signal}")


=== sh.601888 (2025-04-25) 技术指标分析 ===

MACD指标:
MACD值: 0.3947
MACD信号线: 0.7647
MACD柱状图: -0.3700

KDJ指标:
K值: 14.9743
D值: 27.3494
J值: -9.7760

BOLL指标:
上轨: 70.4452
中轨: 63.7595
下轨: 57.0738

=== 技术分析 ===
MACD分析: MACD处于下降趋势
KDJ分析: KDJ超卖
BOLL分析: 股价在布林带中轨运行，趋势盘整


## 计算

In [25]:
"""
五粮液为例
sz.000858
"""
stock_data = df_all_stocks[df_all_stocks["code"] == "sh.601888"]
stock_data["date"] = pd.to_datetime(stock_data["date"])
stock_data = stock_data.sort_values(['code', 'date'])

# 将价格相关的列转换为浮点数
numeric_columns = ['open', 'high', 'low', 'close', 'preclose', 'volume', 'amount', 'turn']
for col in numeric_columns:
    stock_data[col] = pd.to_numeric(stock_data[col])

hs300_constituents = df_hs300[df_hs300["code"] == "sh.601888"]
hs300_constituents['updateDate'] = pd.to_datetime(hs300_constituents['updateDate'])

stock_data = pd.merge(stock_data, hs300_constituents[['code', 'code_name']], on='code', how='inner')

strategy = TradingStrategy()

prepared_data = strategy.prepare_data(stock_data)

signals = strategy.find_trading_signals(prepared_data)



if len(signals) > 0:
    # 添加股票名称
    signals = pd.merge(signals, hs300_constituents[['code', 'code_name']], on='code', how='left')
    
    # 格式化日期
    signals['D1日期'] = signals['D1日期'].dt.strftime('%Y-%m-%d')
    signals['D2日期'] = signals['D2日期'].dt.strftime('%Y-%m-%d')
    
    # 格式化数值列
    numeric_columns = ['D1收盘价', 'D2收盘价', 'D1-D2收益率', 
                        'D1_5日均线', 'D1_20日均线', 'D1_60日均线',
                        'D1_J值', 'D2_J值']
    signals[numeric_columns] = signals[numeric_columns].round(2)
    
    # 设置显示列顺序
    display_columns = ['code', 'code_name', 'D1日期', 'D2日期', 
                        'D1收盘价', 'D2收盘价', 'D1-D2收益率', 
                        'D1_5日均线', 'D1_20日均线', 'D1_60日均线',
                        'D1_J值', 'D2_J值', '持仓天数']
    
    # 保存结果到CSV
    output_path = "trading_signals_ma20.csv"
    signals[display_columns].to_csv(output_path, index=False, encoding='utf-8-sig')
    print(f"\n交易信号明细已保存至: {os.path.abspath(output_path)}")
    
    # 打印统计信息
    print("\n=== 策略回测结果 ===")
    print(f"找到的交易信号总数: {len(signals)}")
    
    print("\n=== 收益率统计 ===")
    print(f"平均收益率: {signals['D1-D2收益率'].mean():.2f}%")
    print(f"收益率中位数: {signals['D1-D2收益率'].median():.2f}%")
    print(f"胜率: {(signals['D1-D2收益率'] > 0).mean() * 100:.2f}%")
    print(f"平均持仓天数: {signals['持仓天数'].mean():.1f}天")
    
    print("\n=== 信号时间分布 ===")
    signals_by_month = signals.groupby(pd.to_datetime(signals['D1日期']).dt.to_period('M')).size()
    print("\n每月信号数量:")
    print(signals_by_month)
    
    print("\n每个交易信号的详细信息:")
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', None)
    print(signals[display_columns].to_string(index=False))
else:
    print("\n未找到符合条件的交易信号")



交易信号明细已保存至: /home/kennys/experiment/QuantTrading/notebook/trading_signals_ma20.csv

=== 策略回测结果 ===
找到的交易信号总数: 1

=== 收益率统计 ===
平均收益率: 5.65%
收益率中位数: 5.65%
胜率: 100.00%
平均持仓天数: 5.0天

=== 信号时间分布 ===

每月信号数量:
D1日期
2024-10    1
Freq: M, dtype: int64

每个交易信号的详细信息:
     code code_name       D1日期       D2日期  D1收盘价  D2收盘价  D1-D2收益率  D1_5日均线  D1_20日均线  D1_60日均线  D1_J值  D2_J值  持仓天数
sh.601888      中国中免 2024-10-17 2024-10-22  65.44  69.14      5.65    68.71     64.41     63.81 -15.92  14.77     5


/tmp/ipykernel_997380/4187777480.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data["date"] = pd.to_datetime(stock_data["date"])
/tmp/ipykernel_997380/4187777480.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs300_constituents['updateDate'] = pd.to_datetime(hs300_constituents['updateDate'])


In [26]:
import pandas as pd
import numpy as np
import talib

# 1. 获取股票数据
stock_data = df_stock_2025[df_stock_2025["code"] == "sh.601127"].copy()
stock_data["date"] = pd.to_datetime(stock_data["date"])
stock_data = stock_data.sort_values('date').reset_index(drop=True)  # 重置索引，确保索引连续

# 2. 将价格相关的列转换为浮点数
numeric_columns = ['open', 'high', 'low', 'close', 'preclose', 'volume', 'amount', 'turn']
for col in numeric_columns:
    stock_data[col] = pd.to_numeric(stock_data[col])

# 3. 计算MACD指标
macd, macd_signal, macd_hist = talib.MACD(
    stock_data['close'],
    fastperiod=12,
    slowperiod=26,
    signalperiod=9
)

# 4. 计算KDJ指标
def calculate_kdj(data, n=9, m1=3, m2=3):
    data = data.copy()
    
    # 计算RSV
    low_list = data['low'].rolling(window=n, min_periods=1).min()
    high_list = data['high'].rolling(window=n, min_periods=1).max()
    rsv = (data['close'] - low_list) / (high_list - low_list) * 100
    
    # 计算K、D、J值
    k = np.zeros(len(data))
    d = np.zeros(len(data))
    
    # 初始化第一个值
    k[0] = 50
    d[0] = 50
    
    # 计算其他值
    for i in range(1, len(data)):
        k[i] = (m1 - 1) * k[i-1] / m1 + rsv[i] / m1
        d[i] = (m2 - 1) * d[i-1] / m2 + k[i] / m2
    
    # 计算J值
    j = 3 * k - 2 * d
    
    return pd.Series(k), pd.Series(d), pd.Series(j)

# 5. 计算WR指标
def calculate_wr(data, period=14):
    high = data['high'].rolling(window=period).max()
    low = data['low'].rolling(window=period).min()
    close = data['close']
    
    # 计算WR值
    wr = -100 * (high - close) / (high - low)
    return wr

# 计算各项指标
k, d, j = calculate_kdj(stock_data)
wr_14 = calculate_wr(stock_data, 14)  # 14日WR
wr_28 = calculate_wr(stock_data, 28)  # 28日WR

# 6. 计算BOLL指标
upper, middle, lower = talib.BBANDS(
    stock_data['close'],
    timeperiod=20,
    nbdevup=2,
    nbdevdn=2,
    matype=0
)

# 7. 将计算结果添加到数据框中
stock_data['MACD'] = macd
stock_data['MACD_signal'] = macd_signal
stock_data['MACD_hist'] = macd_hist
stock_data['KDJ_K'] = k
stock_data['KDJ_D'] = d
stock_data['KDJ_J'] = j
stock_data['WR_14'] = wr_14
stock_data['WR_28'] = wr_28
stock_data['BOLL_upper'] = upper
stock_data['BOLL_middle'] = middle
stock_data['BOLL_lower'] = lower

# 8. 获取2025/4/23的指标值
target_date = '2025-04-25'
result = stock_data[stock_data['date'] == target_date]

if len(result) == 0:
    print(f"未找到 {target_date} 的数据")
else:
    result = result.iloc[0]
    
    # 9. 打印结果
    print(f"\n=== {result['code']} ({target_date}) 技术指标分析 ===")
    print("\nMACD指标:")
    print(f"MACD值: {result['MACD']:.4f}")
    print(f"MACD信号线: {result['MACD_signal']:.4f}")
    print(f"MACD柱状图: {result['MACD_hist']:.4f}")

    print("\nKDJ指标:")
    print(f"K值: {result['KDJ_K']:.4f}")
    print(f"D值: {result['KDJ_D']:.4f}")
    print(f"J值: {result['KDJ_J']:.4f}")

    print("\nWR指标:")
    print(f"WR(14): {result['WR_14']:.4f}")
    print(f"WR(28): {result['WR_28']:.4f}")

    print("\nBOLL指标:")
    print(f"上轨: {result['BOLL_upper']:.4f}")
    print(f"中轨: {result['BOLL_middle']:.4f}")
    print(f"下轨: {result['BOLL_lower']:.4f}")

    # 10. 技术分析
    print("\n=== 技术分析 ===")

    # MACD分析
    if result['MACD_hist'] > 0:
        macd_signal = "MACD处于上升趋势"
    else:
        macd_signal = "MACD处于下降趋势"
    print(f"MACD分析: {macd_signal}")

    # KDJ分析
    if result['KDJ_J'] > 100:
        kdj_signal = "KDJ超买"
    elif result['KDJ_J'] < 0:
        kdj_signal = "KDJ超卖"
    else:
        if result['KDJ_J'] > result['KDJ_K'] and result['KDJ_K'] > result['KDJ_D']:
            kdj_signal = "KDJ金叉，看多信号"
        elif result['KDJ_J'] < result['KDJ_K'] and result['KDJ_K'] < result['KDJ_D']:
            kdj_signal = "KDJ死叉，看空信号"
        else:
            kdj_signal = "KDJ处于盘整区间"
    print(f"KDJ分析: {kdj_signal}")

    # WR分析
    def analyze_wr(wr_value):
        if wr_value > -20:
            return "超买区域"
        elif wr_value < -80:
            return "超卖区域"
        else:
            return "中性区域"
    
    wr_14_signal = analyze_wr(result['WR_14'])
    wr_28_signal = analyze_wr(result['WR_28'])
    print(f"WR(14)分析: {wr_14_signal}")
    print(f"WR(28)分析: {wr_28_signal}")

    # BOLL分析
    close = result['close']
    if close > result['BOLL_upper']:
        boll_signal = "股价突破布林上轨，超买区间"
    elif close < result['BOLL_lower']:
        boll_signal = "股价突破布林下轨，超卖区间"
    else:
        boll_signal = "股价在布林带中轨运行，趋势盘整"
    print(f"BOLL分析: {boll_signal}")

    # 11. 综合分析
    print("\n=== 综合分析 ===")
    signals = []
    
    # 汇总各指标信号
    if result['WR_14'] < -80 and result['WR_28'] < -80:
        signals.append("WR指标显示强烈超卖")
    elif result['WR_14'] > -20 and result['WR_28'] > -20:
        signals.append("WR指标显示强烈超买")
        
    if result['KDJ_J'] < 0:
        signals.append("KDJ指标显示超卖")
    elif result['KDJ_J'] > 100:
        signals.append("KDJ指标显示超买")
        
    if close < result['BOLL_lower']:
        signals.append("布林带显示超卖")
    elif close > result['BOLL_upper']:
        signals.append("布林带显示超买")
        
    # 输出综合分析结果
    if signals:
        print("综合信号:", " | ".join(signals))
    else:
        print("各指标显示市场处于盘整状态")


=== sh.601127 (2025-04-25) 技术指标分析 ===

MACD指标:
MACD值: 0.9611
MACD信号线: 0.9223
MACD柱状图: 0.0389

KDJ指标:
K值: 27.6420
D值: 46.1914
J值: -9.4569

WR指标:
WR(14): -37.9443
WR(28): -34.4613

BOLL指标:
上轨: 137.4816
中轨: 127.8700
下轨: 118.2584

=== 技术分析 ===
MACD分析: MACD处于上升趋势
KDJ分析: KDJ超卖
WR(14)分析: 中性区域
WR(28)分析: 中性区域
BOLL分析: 股价在布林带中轨运行，趋势盘整

=== 综合分析 ===
综合信号: KDJ指标显示超卖


In [17]:
df_hs300[df_hs300["code_name"] == "国电南瑞"]

,updateDate,code,code_name
43,2025-04-21,sh.600406,国电南瑞


In [22]:
df_hs300[df_hs300["code_name"] == "中国中免"]

,updateDate,code,code_name
143,2025-04-21,sh.601888,中国中免


In [16]:
"""
宁德
sz.300750
"""
stock_data = df_all_stocks[df_all_stocks["code"] == "sh.600406"]
stock_data["date"] = pd.to_datetime(stock_data["date"])
stock_data = stock_data.sort_values(['code', 'date'])

# 将价格相关的列转换为浮点数
numeric_columns = ['open', 'high', 'low', 'close', 'preclose', 'volume', 'amount', 'turn']
for col in numeric_columns:
    stock_data[col] = pd.to_numeric(stock_data[col])

hs300_constituents = df_hs300[df_hs300["code"] == "sh.600406"]
hs300_constituents['updateDate'] = pd.to_datetime(hs300_constituents['updateDate'])

stock_data = pd.merge(stock_data, hs300_constituents[['code', 'code_name']], on='code', how='inner')

strategy = TradingStrategy()

prepared_data = strategy.prepare_data(stock_data)

signals = strategy.find_trading_signals(prepared_data)



if len(signals) > 0:
    # 添加股票名称
    signals = pd.merge(signals, hs300_constituents[['code', 'code_name']], on='code', how='left')
    
    # 格式化日期
    signals['D1日期'] = signals['D1日期'].dt.strftime('%Y-%m-%d')
    signals['D2日期'] = signals['D2日期'].dt.strftime('%Y-%m-%d')
    
    # 格式化数值列
    numeric_columns = ['D1收盘价', 'D2收盘价', 'D1-D2收益率', 
                        'D1_5日均线', 'D1_20日均线', 'D1_60日均线',
                        'D1_J值', 'D2_J值']
    signals[numeric_columns] = signals[numeric_columns].round(2)
    
    # 设置显示列顺序
    display_columns = ['code', 'code_name', 'D1日期', 'D2日期', 
                        'D1收盘价', 'D2收盘价', 'D1-D2收益率', 
                        'D1_5日均线', 'D1_20日均线', 'D1_60日均线',
                        'D1_J值', 'D2_J值', '持仓天数']
    
    # 保存结果到CSV
    # output_path = "trading_signals_ma20.csv"
    # signals[display_columns].to_csv(output_path, index=False, encoding='utf-8-sig')
    # print(f"\n交易信号明细已保存至: {os.path.abspath(output_path)}")
    
    # 打印统计信息
    print("\n=== 策略回测结果 ===")
    print(f"找到的交易信号总数: {len(signals)}")
    
    print("\n=== 收益率统计 ===")
    print(f"平均收益率: {signals['D1-D2收益率'].mean():.2f}%")
    print(f"收益率中位数: {signals['D1-D2收益率'].median():.2f}%")
    print(f"胜率: {(signals['D1-D2收益率'] > 0).mean() * 100:.2f}%")
    print(f"平均持仓天数: {signals['持仓天数'].mean():.1f}天")
    
    print("\n=== 信号时间分布 ===")
    signals_by_month = signals.groupby(pd.to_datetime(signals['D1日期']).dt.to_period('M')).size()
    print("\n每月信号数量:")
    print(signals_by_month)
    
    print("\n每个交易信号的详细信息:")
    pd.set_option('display.max_rows', None)
    pd.set_option('display.width', None)
    print(signals[display_columns].to_string(index=False))
else:
    print("\n未找到符合条件的交易信号")



未找到符合条件的交易信号


/tmp/ipykernel_997380/3555849763.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_data["date"] = pd.to_datetime(stock_data["date"])
/tmp/ipykernel_997380/3555849763.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hs300_constituents['updateDate'] = pd.to_datetime(hs300_constituents['updateDate'])


## 前两年的数据

In [ ]:
# 登录系统
lg = bs.login()
if lg.error_code != '0':
    print(f'登录失败，错误代码：{lg.error_code}, 错误信息：{lg.error_msg}')
else:
    print('登录成功')

# 设置日期范围
start_date = '2023-01-01'
end_date = '2023-12-31'

# 初始化存储数据的列表
data_2023 = []

# 遍历每只成分股
for code in df_hs300['code']:
    # 获取日K线数据
    rs = bs.query_history_k_data_plus(
        code,
        "date,code,open,high,low,close,preclose,volume,amount,turn",
        start_date=start_date, end_date=end_date,
        frequency="d", adjustflag="3"
    )
    if rs.error_code != '0':
        print(f'查询股票 {code} 数据失败，错误代码：{rs.error_code}, 错误信息：{rs.error_msg}')
        continue

    # 保存数据
    stock_data = []
    while rs.next():
        stock_data.append(rs.get_row_data())
    df_stock = pd.DataFrame(stock_data, columns=rs.fields)
    data_2023.append(df_stock)

# 合并所有股票的数据
df_all_stocks = pd.concat(data_2023, ignore_index=True)

# 登出系统
bs.logout()

In [ ]:
df_all_stocks